<a href="https://colab.research.google.com/github/nguyen084/MATH5671/blob/master/MATH5671_Group005_Assigment009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Drive mounting

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Something is wrong with Spark 2.4.4 and Java 8. Should use lower version 2.2.3. Also only Java 8 works. 

In [0]:
!wget -q https://archive.apache.org/dist/spark/spark-2.2.3/spark-2.2.3-bin-hadoop2.7.tgz
!tar xf spark-2.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
!ls

gdrive	sample_data  spark-2.2.3-bin-hadoop2.7	spark-2.2.3-bin-hadoop2.7.tgz


Install Java 8 if Java 11 was installed. Spark is compatible with Java 8 now

In [13]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


Switch to Java 8 if it is Java 11 now

In [14]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2


Double check if it is Java 8 now

In [15]:
!java -version 

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


Set some environment variables

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [18]:
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
spark = SparkSession(sc)

ValueError: ignored

Reading Data

In [0]:
pathdir ='/content/gdrive/My Drive/Academic/UCONN/Financial Research/MATH5671/'

In [20]:
#Load dataframe#
payment = spark.read.format("csv").option("header", "true").load(pathdir+'payment.csv').alias("payment").createOrReplaceTempView("payment")
rental = spark.read.format("csv").option("header", "true").load(pathdir+'rental.csv').alias('rental').createOrReplaceTempView("rental")
staff = spark.read.format("csv").option("header", "true").load(pathdir+'staff.csv').alias('staff').createOrReplaceTempView("staff")
inventory = spark.read.format("csv").option("header", "true").load(pathdir+'inventory.csv').alias('inventory').createOrReplaceTempView("inventory")
store = spark.read.format("csv").option("header", "true").load(pathdir+'store.csv').alias('store').createOrReplaceTempView("store")
address = spark.read.format("csv").option("header", "true").load(pathdir+'address.csv').alias('address').createOrReplaceTempView("address")
customer = spark.read.format("csv").option("header", "true").load(pathdir+'customer.csv').alias('customer').createOrReplaceTempView("customer")
film = spark.read.format("csv").option("header", "true").load(pathdir+'film.csv').alias('film').createOrReplaceTempView("film")

print('Question 1: Payment DataFrame')
spark.sql("select customer_id, sum(amount) as total_amount from payment group by customer_id order by total_amount DESC").show(3)

print('Question 2: Rental Dataframe')
spark.sql("select * from rental where year(rental_date) = 2005 order by return_date").show(3)

print('Question 3: Count Staff ID')
spark.sql("select staff_id, count(rental_id) as count from rental group by staff_id").show()

print('Question 4: Count by first and last name')
spark.sql("select first(first_name) as first_name, first(last_name) as last_name, count(rental_id) as count from staff inner join rental on rental.staff_id = staff.staff_id group by staff.staff_id").show()

print('Question 5:')
spark.sql("""select rental.rental_date, rental.return_date, address.address, film.title, film.description, customer.first_name, customer.last_name, staff.first_name, staff.last_name from rental join inventory on rental.inventory_id = inventory.inventory_id join store on inventory.store_id = store.store_id join address on store.address_id = address.address_id join customer on rental.customer_id = customer.customer_id join staff on staff.store_id = customer.store_id join film on inventory.film_id = film.film_id order by rental.rental_date""").show(3)

print('Question 6:')
spark.sql("""select customer_id, total_amount from (select customer_id, sum(amount) as total_amount from payment where year(payment_date) =2005 group by customer_id)
where total_amount > 200""").show()

print('Question 7:')
spark.sql("""select * from (select first(customer.first_name) as first_name, first(customer.last_name) as last_name, sum(amount) as total_amount from payment inner join customer on customer.customer_id = payment.customer_id where year(payment_date) =2005 group by payment.customer_id) where total_amount > 200 """).show()

print('Question 8:')
year2005 = spark.sql("select customer_id, year(rental_date) as rentdate from rental where year(rental_date) = 2005").createOrReplaceTempView("year2005")
year2006 = spark.sql("select customer_id, year(rental_date) as rentdate from rental where year(rental_date) = 2006").createOrReplaceTempView("year2006")
spark.sql("select customer_id, year(rental_date) as rentdate from rental where year(rental_date) = 2005").show(3)
spark.sql("select customer_id, year(rental_date) as rentdate from rental where year(rental_date) = 2006").show(3)
spark.sql("""select count(distinct(year2005.customer_id)) as customer_id_in_2005_not_2006 from year2005 where year2005.customer_id not in (select year2006.customer_id from year2006) """).show()
spark.sql("""select count(year2006.customer_id) as customer_id_in_2006_not_2005 from year2006 where year2006.customer_id not in ( select year2005.customer_id from year2005) """).show()

Question 1: Payment DataFrame
+-----------+------------------+
|customer_id|      total_amount|
+-----------+------------------+
|        526| 221.5500000000001|
|        148| 216.5400000000001|
|        144|195.58000000000007|
+-----------+------------------+
only showing top 3 rows

Question 2: Rental Dataframe
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+
|       32|2005-05-25 04:06:21|        3832|        230|2005-05-25 23:55:21|       1|2006-02-15 21:30:53|
|       21|2005-05-25 01:59:46|         146|        388|2005-05-26 01:01:46|       2|2006-02-15 21:30:53|
|       14|2005-05-25 00:31:15|        2701|        446|2005-05-26 02:56:15|       1|2006-02-15 21:30:53|
+---------+-------------------+------------+-----